In [1]:
import sys
import types
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import grid_search
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


C:\Users\loli\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\loli\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
df = pd.read_csv('dataset/creditcard.csv')

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.shape

(284807, 31)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

In [6]:
pd.set_option('precision', 3)
df.loc[:, ['Time', 'Amount']].describe()

,Time,Amount
count,284807.000,284807.000
mean,94813.860,88.350
std,47488.146,250.120
min,0.000,0.000
25%,54201.500,5.600
50%,84692.000,22.000
75%,139320.500,77.165
max,172792.000,25691.160


In [27]:
#skewness
skew_ = df.skew()
skew_

Time      -0.036
V1        -3.281
V2        -4.625
V3        -2.240
V4         0.676
V5        -2.426
V6         1.827
V7         2.554
V8        -8.522
V9         0.555
V10        1.187
V11        0.357
V12       -2.278
V13        0.065
V14       -1.995
V15       -0.308
V16       -1.101
V17       -3.845
V18       -0.260
V19        0.109
V20       -2.037
V21        3.593
V22       -0.213
V23       -5.875
V24       -0.552
V25       -0.416
V26        0.577
V27       -1.170
V28       11.192
Amount    16.978
Class     23.998
dtype: float64

# Scaling Amount and Time

In [7]:
scaler = StandardScaler()
scaler2 = StandardScaler()
#scaling time
scaled_time = scaler.fit_transform(df[['Time']])
flat_list1 = [item for sublist in scaled_time.tolist() for item in sublist]
scaled_time = pd.Series(flat_list1)

In [8]:
#scaling the amount column
scaled_amount = scaler2.fit_transform(df[['Amount']])
flat_list2 = [item for sublist in scaled_amount.tolist() for item in sublist]
scaled_amount = pd.Series(flat_list2)

In [9]:
#concatenating newly created columns w original df
df = pd.concat([df, scaled_amount.rename('scaled_amount'), scaled_time.rename('scaled_time')], axis=1)
df.sample(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,scaled_amount,scaled_time
53564,46047.0,-0.685,0.743,0.990,-1.251,0.781,-0.990,1.081,-0.135,-0.965,...,-0.338,0.065,0.298,0.916,-0.072,0.055,8.17,0,-0.321,-1.027
41702,40792.0,-0.739,0.876,1.268,-0.103,0.634,-0.587,1.157,0.009,-0.923,...,0.007,-0.105,0.035,-0.880,0.045,0.100,45.55,0,-0.171,-1.138
221315,142522.0,-1.106,1.078,1.637,1.159,-0.332,0.007,0.057,0.711,0.169,...,-0.095,-0.136,0.333,-0.772,0.226,0.056,4.37,0,-0.336,1.005
93812,64595.0,1.145,-0.092,0.601,0.727,-0.403,0.185,-0.322,0.189,0.373,...,-0.068,0.076,0.480,0.452,-0.003,-0.004,3.79,0,-0.338,-0.636
40032,40066.0,-2.615,-0.880,0.762,0.279,1.292,-0.191,1.856,-0.712,-0.125,...,1.032,-0.376,-0.100,-0.965,-0.848,0.359,126.15,0,0.151,-1.153


In [10]:
#dropping old amount and time columns
df.drop(['Amount', 'Time'], axis=1, inplace=True)

# Splitting Data into Train and Test


In [11]:
X = df.drop('Class', 1)
y = df['Class']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build model

In [13]:
rf_model = RandomForestClassifier(
    n_estimators=1,
    criterion='gini',
    max_depth=7,
    min_samples_split=2,
    min_samples_leaf=5,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=16,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None).fit(X_train, y_train)


# Make predictions

In [14]:
pred_train = rf_model.predict(X_train)
pred_test = rf_model.predict(X_test)

# Evaluate model

### Accuracy

In [15]:
print("Train accuracy:" + str(rf_model.score(X_train, y_train)))
print("Test accuracy:" + str(rf_model.score(X_test, y_test)))

Train accuracy:0.9994031029866796
Test accuracy:0.9992802219023208


### Confusion matrix - train

In [16]:
confusion_matrix(y_train, pred_train)

array([[227428,     23],
       [   113,    281]], dtype=int64)

### Confusion matrix - test

In [17]:
confusion_matrix(y_test, pred_test)

array([[56857,     7],
       [   34,    64]], dtype=int64)

### Precision and Recall - train

In [18]:
precision, recall,_= precision_recall_curve(y_train, pred_train)
print('Precision: '+ str(precision[1]))
print('Recall: '+ str(recall[1]))

Precision: 0.9243421052631579
Recall: 0.7131979695431472


### Precision and Recall - test

In [19]:
precision, recall,_= precision_recall_curve(y_test, pred_test)
print('Precision: '+ str(precision[1]))
print('Recall: '+ str(recall[1]))

Precision: 0.9014084507042254
Recall: 0.6530612244897959


# Improve results - through parameter optimization

### select model type

In [20]:
rf_model = RandomForestClassifier(
    n_estimators=1,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=5,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=16,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=None)

### Define grid

In [21]:
param_grid={
    'n_estimators' : [6, 7, 8, 9, 10, 20, 30],
    'max_depth' : [7, 8, 9, 10, 11]
}

### Define cross validation and scoring

In [22]:
clf = grid_search.GridSearchCV(
    rf_model,
    param_grid,
    n_jobs=16,
    cv=3,
scoring='recall')

### Search for better parameter

In [23]:
### time
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=16,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=16,
       param_grid={'n_estimators': [6, 7, 8, 9, 10, 20, 30], 'max_depth': [7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, scoring='recall', verbose=0)

### Evaluate best model

In [24]:
rf_best = clf.best_estimator_

In [25]:
rf_best

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=11, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=6, n_jobs=16,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [26]:
pred_train = rf_best.predict(X_train)
pred_test = rf_best.predict(X_test)


In [27]:
confusion_matrix(y_train, pred_train)

array([[227436,     15],
       [    83,    311]], dtype=int64)

In [28]:
confusion_matrix(y_test, pred_test)

array([[56855,     9],
       [   24,    74]], dtype=int64)

In [29]:
precision, recall,_= precision_recall_curve(y_train, pred_train)
print('Precision: '+ str(precision[1]))
print('Recall: '+ str(recall[1]))

Precision: 0.9539877300613497
Recall: 0.7893401015228426


In [30]:
precision, recall,_= precision_recall_curve(y_test, pred_test)
print('Precision: '+ str(precision[1]))
print('Recall: '+ str(recall[1]))

Precision: 0.891566265060241
Recall: 0.7551020408163265
